In [264]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [277]:
df = pd.read_csv(dirname + '/train.csv')
df2 = pd.read_csv(dirname + '/test.csv')
pd.options.mode.chained_assignment = None
df2.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [278]:
# create train_x and train_y
train_y = np.array([df["Survived"].values])

columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
data = df[columns]
# male -> 0, femail -> 1
data.replace({'male': 0, 'female': 1}, inplace=True)
data['Fare'].fillna(int(data['Fare'].mean()), inplace=True)
data['Age'].fillna(int(data['Age'].mean()), inplace=True)
data = data.to_numpy().astype(float)
for i in range(data.shape[1]): 
    data[:, i] = (data[:, i] - data[:, i].mean()) / data[:, i].std()
train_x = data.T
data = df2[columns]
# male -> 0, femail -> 1
data.replace({'male': 0, 'female': 1}, inplace=True)
data['Fare'].fillna(int(data['Fare'].mean()), inplace=True)
data['Age'].fillna(int(data['Age'].mean()), inplace=True)
data = data.to_numpy().astype(float)
for i in range(data.shape[1]): 
    data[:, i] = (data[:, i] - data[:, i].mean()) / data[:, i].std()
test_x = data.T

In [279]:
print(f'train_x: {train_x.shape}, train_y: {train_y.shape}, test_x: {test_x.shape}')

train_x: (6, 891), train_y: (1, 891), test_x: (6, 418)


In [280]:
# sigmoid function 
def sigmoid(z): 
    s = 1 / (1 + np.exp(-z))
    return s

In [281]:
# initialize parameters
def initialize_parameters(dim): 
    W = np.zeros((1, dim))
    b = 0 
    return W, b 

In [282]:
# forward propagate & backward propagate (W, b, X, Y)
def propagate(W, b, X, Y): 
    m = X.shape[1]
    A = sigmoid(np.dot(W, X) + b)
    cost = -1/m * np.sum( np.dot(np.log(A), Y.T) + np.dot(np.log(1-A), (1-Y.T)))
   # print(f'cost here is {cost}')
    dW = 1/m * np.dot((A-Y), X.T)
    db = 1/m * np.sum(A-Y, axis = 1, keepdims = True)
    
    grads = {'dW': dW, 'db': db}
    
    return grads, cost

In [294]:
def optimize(W, b, X, Y, num_iterations, lr, print_cost = False): 
    costs = []
    for i in range(num_iterations): 
        grads, cost = propagate (W, b, X, Y)
        dW = grads['dW']
        db = grads['db']
        W = W - lr * dW
        b = b - lr * db
        if i % 1000 == 0: 
            costs.append(cost)
        if print_cost and i % 1000 == 0: 
            print(f"Cost after iteration {i}: {cost}")
            
    params = {"W": W, 'b': b}
    grads = {"dW": dW, 'db': db}
        
    return params, grads, costs

In [295]:
w, b, X, Y = np.array([[1.,2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dW"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[0.99845601 2.39507239]]
db = [[0.00145558]]
cost = 5.801545319394553


In [296]:
b

2.0

In [297]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 1000, lr = 0.009, print_cost = False)

print ("w = " + str(params["W"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dW"]))
print ("db = " + str(grads["db"]))
costs

w = [[-0.64226437 -0.43498153]]
b = [[2.20255947]]
dw = [[ 0.06282959 -0.01416124]]
db = [[-0.04847509]]


[5.801545319394553]

In [298]:
def model(train_x, train_y, test_x, num_iterations, lr, print_cost=False): 
    W, b = initialize_parameters(train_x.shape[0])
    parameters, grads, costs = optimize(W, b, train_x, train_y, num_iterations, lr, True)
    W = parameters["W"]
    b = parameters["b"]
    
    test_y = (sigmoid(np.dot(W, test_x) + b) > 0.5).astype(int)
    return test_y
    

In [303]:
test_y = model(train_x, train_y, test_x, 4000, 0.5, True)

Cost after iteration 0: 0.693147180559945
Cost after iteration 1000: 0.4427003155643808
Cost after iteration 2000: 0.4427003155643809
Cost after iteration 3000: 0.4427003155643809


In [306]:
test_y.shape

(1, 418)

In [316]:
df3 = pd.DataFrame(df2['PassengerId'])
df3["Survived"] = test_y.tolist()[0]

In [319]:
df3.to_csv('submission_logistic.csv')

In [320]:
df3.shape

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
